In [ ]:
!pip install langchain
!pip install openai
!pip install neo4j
!pip3 install python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created whee

In [ ]:
import os
import json
import pandas as pd

from openai import AzureOpenAI  #pip install openai
import dotenv

dotenv.load_dotenv("openai_azure_example.env",override=True)

resource_name = os.environ.get("AZURE_RESOURCE_NAME")
chat_deployment_name=os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
embedding_deployment_name=os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key=os.environ.get("AZURE_OPENAI_API_KEY")
# openai_api_key=os.environ.get("AZURE_OPENAI_API_KEY")
api_version="2023-12-01-preview"  # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference
endpoint=f"https://{resource_name}.openai.azure.com"
url=f"https://{resource_name}.openai.azure.com/openai/deployments/{chat_deployment_name}/chat/completions?api-version={api_version}"




In [ ]:
url = "neo4j+s://5b1be4ed.databases.neo4j.io"
username ="neo4j"
password = "eJyOK3UNTyPEBdhh5pt05jSSALxENqSUhnGcEpuCJiw"

In [ ]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"

In [ ]:
entity_types = {
    "chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
    "subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
    "part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
    "subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
    "section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}
relation_types = {
    "HAS_SUBCHAPTER": "A chapter contains zero or more subchapters.",
    "HAS_PART": "A subchapter contains one or more parts.",
    "HAS_SUBPART": "A part contains one or more subparts.",
    "HAS_SECTION": "A subpart contains one or more sections."
}
entity_relationship_match = {
    "chapter": "HAS_SUBCHAPTER",
    "subchapter": "HAS_PART",
    "part": "HAS_SUBPART",
    "subpart": "HAS_SECTION"
}

In [ ]:
system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {json.dumps(entity_types, indent=0)}

    Each entity is connected through one of the following hierarchical relationships:
    {json.dumps(relation_types, indent=0)}

    Depending on the user prompt, determine if it is possible to answer with the graph database.

    The graph database can navigate through multiple layers of hierarchy to find specific sections of regulations.

    Example user input:
    "We have a continuous emission monitoring system for our acid gas units but no vent meter, what calculation method should we use?"

    There are multiple layers to analyse:
    1. The mention of "continuous emission monitoring system" indicates what subject matter the prompt is asking for.
    2. The mention of "calculation method" provides the clarification on the subject matter.
    3. The mention of "no vent meter" provides additional conditions to the subject matter.


    Return a json object following these rules:
    For each layer of the hierarchy or specific query parameter mentioned, add a key-value pair with the key being a match for one of the entity types provided, and the value being the relevant detail from the user query.

    For the example provided, the expected output would be:
    {{
        "subject" : "CEMS for our acid gas units",
        "subject_interest" : "calculation method",
        "condition" : "no vent meter"
    }}

    If there are no relevant entities or layers in the user prompt, return an empty json object.
'''

print(system_prompt)


    You are a helpful agent designed to fetch information from a graph database structured around environmental regulations.

    The graph database organizes regulations into the following entity types:
    {
"chapter": "Top-level division, generalizing a broad area of the subject that are presented in segment text.",
"subchapter": "Subdivision of a chapter, focusing on a more specific area within the chapter's scope often naming specific agency and government act.",
"part": "Segment of a subchapter, detailing more specific topics, regulations, agencies and guidelines.",
"subpart": "Further division of a part, detailing very specific aspects or regulations and topic of interest.",
"section": "The most granular division, often representing specific area in individual regulations, guidelines, or information pieces, containing text, formulas and extractions to answer regulatory questions."
}

    Each entity is connected through one of the following hierarchical relationships:
    {
"HA

In [ ]:

client = AzureOpenAI(api_key=api_key,api_version=api_version,azure_endpoint=endpoint)


def define_query(prompt, model="chat35"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format= {
            "type": "json_object"
        },
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion.choices[0].message.content

In [ ]:
example_queries = [
    "What is the emission reporting limit?",
    "Do I have to count the number of pneumatic devices we have?",
    "What are my pneumatic device emissions? I have 100 high bleed devices."
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")

Q: 'What is the emission reporting limit?'
{
    "subject_interest": "emission reporting limit"
}

Q: 'Do I have to count the number of pneumatic devices we have?'
{
    "subject_interest": "count the number of pneumatic devices"
}

Q: 'What are my pneumatic device emissions? I have 100 high bleed devices.'
{
    "subject": "pneumatic device emissions",
    "condition": "100 high bleed devices"
}



In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"

In [ ]:
def create_embedding(text):
    result = client.embeddings.create(model=embedding_deployment_name, input=text)
    return result.data[0].embedding

In [ ]:
def create_query(text, threshold=0.81):
    # Assume `text` contains JSON with keys corresponding to entity types (e.g., "chapter", "section")
    # and their desired attributes or content for query.
    query_data = json.loads(text)
    print(query_data)

    # Creating embeddings part of the query might be less relevant unless you are comparing text content or attributes
    # For simplicity, this example focuses on structuring a Cypher query to navigate the hierarchy based on provided criteria.

    # Start constructing the Cypher query
    # This example assumes you want to find sections meeting certain criteria
    query_parts = []
    for key, val in query_data.items():
        #if key in ["subject","subject_interest","Chapter", "Subchapter", "Part", "Subpart", "Section"]:
        if key in ["subject"]:
            # Assuming you have a way to directly filter nodes based on attributes or content
            query_parts.append(f"MATCH (n:Section) WHERE n.text CONTAINS '{val}' LIMIT 20;")

    # Constructing the path through the hierarchy
    if query_parts:
        # Assuming a linear path for simplicity; adjust based on your actual data model and query logic
        query = "\n".join(query_parts)
        query += "\nRETURN n"
    else:
        query = "RETURN 'No matching criteria provided.'"

    return query


In [ ]:
def query_graph(response):
    embeddingsParams = {}
    query = create_query(response)
    query_data = json.loads(response)
    for key, val in query_data.items():
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    result = graph.query(query, params=embeddingsParams)
    return result

In [ ]:
example_response = '''{
    "subject" : "CEMS",
    "subject_interest" : "calculation method",
    "condition" : "no vent meter"
}'''

result = query_graph(example_response)

{'subject': 'CEMS', 'subject_interest': 'calculation method', 'condition': 'no vent meter'}


In [ ]:
print(f"Found {len(result)} matching section(s):\n")
for r in result:
    #print(f"{r['p']['name']} ({r['p']['id']})")
    print(r)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

